# Webscrape data


### Imports and functions

In [1]:
import pandas as pd
import os
from pathlib import Path
import time
import requests
from random import uniform
from bs4 import BeautifulSoup
import random
from fake_useragent import UserAgent
import unicodedata

def remove_special_characters(text):
    # Normalize the string to NFKD form
    normalized = unicodedata.normalize('NFKD', text)
    # Remove diacritics (non-spacing marks)
    without_special_chars = ''.join(c for c in normalized if not unicodedata.combining(c))
    return without_special_chars

### Naming convention for the tables

In [2]:
titles_12 = [
    'serie_A_overall',
    'serie_A_homeaway',
    'squad_standard_stats',
    'squad_standard_stats_opp',
    'squad_goalkeeping',
    'squad_goalkeeping_opp',
    'squad_advanced_goalkeeping',
    'squad_advanced_goalkeeping_opp',
    'squad_shooting',
    'squad_shooting_opp',
    'squad_passing',
    'squad_passing_opp',
    'squad_pass_types',
    'squad_pass_types_opp',
    'squad_goal_shot_creation',
    'squad_goal_shot_creation_opp',
    'squad_defensive_actions',
    'squad_defensive_actions_opp',
    'squad_possession',
    'squad_possession_opp',
    'squad_playing_time',
    'squad_playing_time_opp',
    'squad_miscellaneous',
    'squad_miscellaneous_opp',
]

titles_6 = titles_12[:6] + titles_12[8:10] + titles_12[20:]

titles_5 = titles_6[:6] + titles_6[8:]

### Acquire the league data

In [ ]:
os.makedirs('data/clubes/', exist_ok=True)
os.makedirs('data/league/', exist_ok=True)

In [ ]:
# Fetch all tables from the webpage
os.makedirs('data', exist_ok=True)
for page in range(11):
    
    url = f"https://fbref.com/en/comps/24/{2024 - page}/{2024 - page}-Serie-A-Stats"
    tables = pd.read_html(url)
    year = 2024 - page
    
    # Convert each table to a DataFrame
    if year > 2018:
        dfs = {f"{titles_12[i]}_{year}": table for i, table in enumerate(tables)}
    
    elif year > 2015:
        dfs = {f"{titles_6[i]}_{year}": table for i, table in enumerate(tables)}
    
    else:
        dfs = {f"{titles_5[i]}_{year}": table for i, table in enumerate(tables)}
    
    for name, df in dfs.items():
        df.to_csv(f"data/league/{name}.csv", index=False)


### Acquire player data

#### Get squads that where on Serie A and the year

In [1]:
## ID da página de acesso de cada clube
clubes = {
        'Botafogo-RJ': {'id': 'd9fdd9d9'},
        'Palmeiras': {'id': 'abdce579'},
        'Flamengo': {'id': '639950ae'},
        'Fortaleza': {'id': 'a9d0ab0e'},
        'Internacional': {'id': '6f7e1f03'},
        'Sao-Paulo': {'id': '5f232eb1'},
        'Corinthians': {'id': 'bf4acd28'},
        'Bahia': {'id': '157b7fee'},
        'Cruzeiro': {'id': '03ff5eeb'},
        'Vasco-da-Gama': {'id': '83f55dbe'},
        'Vitoria': {'id': '33f95fe0'},
        'Atletico-Mineiro': {'id': '422bb734'},
        'Fluminense': {'id': '84d9701c'},
        'Gremio': {'id': 'd5ae3703'},
        'Juventude': {'id': 'd081b697'},
        'RB-Bragantino': {'id': 'f98930d1'},
        'Ath-Paranaense': {'id': '2091c619'},
        'Criciuma': {'id': '3f7595bb'},
        'Atl-Goianiense': {'id': '32d508ca'},
        'Cuiaba': {'id': 'f0e6fb14'},
        'Santos': {'id': '712c528f'},
        'Goias': {'id': '78c617cc'},
        'Coritiba': {'id': 'd680d257'},
        'America-MG': {'id': '1f68d780'},
        'Ceara': {'id': '2f335e17'},
        'Avai': {'id': 'f205258a'},
        'Sport-Recife': {'id': 'ece66b78'},
        'Chapecoense': {'id': 'baa296ad'},
        'CSA': {'id': '05aff519'},
        'Parana': {'id': '2091c619'},
        'Ponte-Preta': {'id': 'b162ebe7'},
        'Figueirense': {'id': '0ce4436d'},
        'Santa-Cruz': {'id': 'ad0c1246'},
        'Joinville': {'id': 'da0666a2'},
    }
CLUBES = clubes.keys()

In [3]:
for clube in clubes:
    clubes[clube]['years_SA'] = []

In [4]:
# verifica se o time estava na serie A naquele ano e adiciona o ano na key 'years_SA'

for i in range(11):
    df = pd.read_csv(f'data/league/serie_A_overall_{2024 - i}.csv')

    unique_values = df['Squad'].unique()

    for value in unique_values:
        value = remove_special_characters(value.replace(" ", "-").replace("(", "").replace(")", ""))

        clubes[value]['years_SA'].append(2024 - i)
        

#### Scraping with pd.read_html()

In [5]:
for clube, dados in clubes.items():
    # Create club directory if it doesn't exist
    club_dir = Path(f'data/clubes/{clube}')
    club_dir.mkdir(parents=True, exist_ok=True)
    
    for year in dados['years_SA']:
        # Generate expected filenames pattern for this year
        existing_files = list(club_dir.glob(f"{year}_*.csv"))
        
        # Skip if we already have files for this year
        if existing_files:
            print(f"Skipping {clube} {year} - already downloaded")
            continue
        
        url = f"https://fbref.com/en/squads/{clubes[clube]['id']}/{year}/{clube}-Stats"

        try:
            # Add delay to avoid 429 errors
            time.sleep(uniform(5, 30))
            tables = pd.read_html(url)
            
            # Save each table
            for i, table in enumerate(tables):
                filename = club_dir / f"{year}_{i}.csv"
                table.to_csv(filename, index=False)
            
            print(f"Saved {clube} {year}")
                
        except Exception as e:
            print(f"Failed to scrape {clube} {year}: {str(e)}")
            continue


Skipping Botafogo-RJ 2024 - already downloaded
Skipping Botafogo-RJ 2023 - already downloaded
Skipping Botafogo-RJ 2022 - already downloaded
Skipping Botafogo-RJ 2020 - already downloaded
Skipping Botafogo-RJ 2019 - already downloaded
Skipping Botafogo-RJ 2018 - already downloaded
Skipping Botafogo-RJ 2017 - already downloaded
Skipping Botafogo-RJ 2016 - already downloaded
Skipping Botafogo-RJ 2014 - already downloaded
Skipping Palmeiras 2024 - already downloaded
Skipping Palmeiras 2023 - already downloaded
Skipping Palmeiras 2022 - already downloaded
Skipping Palmeiras 2021 - already downloaded
Skipping Palmeiras 2020 - already downloaded
Skipping Palmeiras 2019 - already downloaded
Skipping Palmeiras 2018 - already downloaded
Skipping Palmeiras 2017 - already downloaded
Skipping Palmeiras 2016 - already downloaded
Skipping Palmeiras 2015 - already downloaded
Skipping Palmeiras 2014 - already downloaded
Skipping Flamengo 2024 - already downloaded
Skipping Flamengo 2023 - already downl

#### Rename the tables

In [17]:
title_clubes_13 = [
    'standard',
    'scores_fixtures',
    'goalkeeping',
    'advanced_goalkeeping',
    'shooting',
    'passing',
    'pass_types',
    'goal_shot_creation',
    'defesensive_actions',
    'possession',
    'playingtime',
    'miscellaneous',
    'serie_A_overall',
    'serie_A_homeaway',
]
title_clubes_7 = title_clubes_13[:3] + [title_clubes_13[4]] + title_clubes_13[10:]


In [ ]:
for clube, dados in clubes.items():
    
    club_dir = Path(f'data/clubes/{clube}')
    
    files = os.listdir(club_dir)
    
    for file in files:
       
        if file.split('_')[-1].split('.')[0].isdigit():
            
            year, digit = file.split('_')
            digit = digit.split('.')[0]

            if int(year) > 2018:
                new_name = f"{year}_{title_clubes_13[int(digit)]}.csv"
            else:
                new_name = f"{year}_{title_clubes_7[int(digit)]}.csv"


            os.rename(club_dir / file, club_dir / new_name)
            #print(club_dir / file, club_dir / new_name)




#### Test scraping

In [10]:
url = f"https://fbref.com/en/squads/d9fdd9d9/2022/Botafogo-RJ-Stats"
    

tables = pd.read_html(url)

# Save each table
for i, table in enumerate(tables):
    filename = f"data/clubes/Botafogo-RJ/{2022}_{i}.csv"
    table.to_csv(filename, index=False)

# Preprocessing

### Imports and functions

In [25]:
import pandas as pd
import os
from pathlib import Path

clubes = {
        'Botafogo-RJ': {'id': 'd9fdd9d9'},
        'Palmeiras': {'id': 'abdce579'},
        'Flamengo': {'id': '639950ae'},
        'Fortaleza': {'id': 'a9d0ab0e'},
        'Internacional': {'id': '6f7e1f03'},
        'Sao-Paulo': {'id': '5f232eb1'},
        'Corinthians': {'id': 'bf4acd28'},
        'Bahia': {'id': '157b7fee'},
        'Cruzeiro': {'id': '03ff5eeb'},
        'Vasco-da-Gama': {'id': '83f55dbe'},
        'Vitoria': {'id': '33f95fe0'},
        'Atletico-Mineiro': {'id': '422bb734'},
        'Fluminense': {'id': '84d9701c'},
        'Gremio': {'id': 'd5ae3703'},
        'Juventude': {'id': 'd081b697'},
        'RB-Bragantino': {'id': 'f98930d1'},
        'Ath-Paranaense': {'id': '2091c619'},
        'Criciuma': {'id': '3f7595bb'},
        'Atl-Goianiense': {'id': '32d508ca'},
        'Cuiaba': {'id': 'f0e6fb14'},
        'Santos': {'id': '712c528f'},
        'Goias': {'id': '78c617cc'},
        'Coritiba': {'id': 'd680d257'},
        'America-MG': {'id': '1f68d780'},
        'Ceara': {'id': '2f335e17'},
        'Avai': {'id': 'f205258a'},
        'Sport-Recife': {'id': 'ece66b78'},
        'Chapecoense': {'id': 'baa296ad'},
        'CSA': {'id': '05aff519'},
        'Parana': {'id': '2091c619'},
        'Ponte-Preta': {'id': 'b162ebe7'},
        'Figueirense': {'id': '0ce4436d'},
        'Santa-Cruz': {'id': 'ad0c1246'},
        'Joinville': {'id': 'da0666a2'},
}

CLUBES = clubes.keys()

def process_names(df):
    """
    Renames columns in the DataFrame based on their first row values,
    drops the first row, removes the 'Matches' column if it exists, 
    and resets the index.

    Parameters:
    df (pd.DataFrame): The input DataFrame to process.

    Returns:
    pd.DataFrame: The processed DataFrame with updated column names, 
                  the first row removed, and the index reset.
    """

    # Create rename dictionary first to avoid modifying DataFrame during iteration
    rename_dict = {}
    for col in df.columns:
        # Use .at[] for faster scalar value access
        first_row_value = df.at[0, col]
        
        if 'Unnamed' in col:
            new_name = first_row_value
        else:
            # Handle potential missing '.' safely
            prefix = col.split('.')[0]
            new_name = f"{prefix} {first_row_value}"
        
        rename_dict[col] = new_name

    # Apply all renames at once
    df = df.rename(columns=rename_dict)
    
    # Drop first row (index 0)
    df = df.drop(index=0)
    
    # Safely drop 'Matches' column if it exists
    if 'Matches' in df.columns:
        df = df.drop(columns=['Matches'])
    
    # Reset index after dropping row
    return df.reset_index(drop=True)

def get_gringos(df, gringos, clube=None, year=None, nation=['br BRA']):
    """
    Filters the provided dataframe and returns only the players 
    that are not from the especified nationality(s).

    Parameters:
    df (pd.DataFrame): The input DataFrame to filter and process.
    gringos (pd.DataFrame): The existing DataFrame to append filtered rows to.
    clube (str, optional): Pass a name to add the squad name to the new df.
    nation (list): The nation(s) to be filtered out, default is 'br BRA'.

    Returns:
    pd.DataFrame: The updated `gringos` DataFrame with filtered rows 
                  appended and the optional 'Clube' column added.
    """

    # Create a copy of the filtered data to avoid SettingWithCopyWarning
    new_gringos = df[~df['Nation'].isin(nation) & ~df['Nation'].isna()].copy()
    
    if clube:
        new_gringos.insert(loc=5, column='Clube', value=clube)
        #new_gringos['Clube'] = clube
    if clube:
        new_gringos.insert(loc=6, column='Year', value=year)
        #new_gringos['Year'] = year

    if gringos.empty:
        gringos = new_gringos

    else:
        gringos = pd.concat([gringos, new_gringos], ignore_index=True)
    
    return gringos

### Testing the functions

In [28]:
gringos_df = pd.DataFrame()

for clube in CLUBES:
    
    club_dir = Path(f'data/clubes/{clube}')
    
    files = os.listdir(club_dir)
    
    for file in files:
        year = file.split('_')[0]
        if "standard" in file: #and year == '2024':
            teste_df = process_names(pd.read_csv(f'{club_dir}/{file}'))
            gringos_df = get_gringos(teste_df, gringos_df, clube=clube, year=year)
            

In [30]:
gringos_df.to_csv('data/processed/gringos_df.csv')

In [23]:
teste_df = process_names(pd.read_csv('data/clubes/Botafogo-RJ/2024_standard.csv'))
gringos_df = pd.DataFrame()

In [24]:
teste_df

,Player,Nation,Pos,Age,Playing Time MP,Playing Time Starts,Playing Time Min,Playing Time 90s,Performance Gls,Performance Ast,...,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes G+A,Per 90 Minutes G-PK,Per 90 Minutes G+A-PK,Per 90 Minutes xG,Per 90 Minutes xAG,Per 90 Minutes xG+xAG,Per 90 Minutes npxG,Per 90 Minutes npxG+xAG
0,John,br BRA,GK,27.0,34,34,3060.0,34.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Luiz Henrique,br BRA,"FW,MF",23.0,35,33,2533.0,28.1,7.0,3.0,...,0.25,0.11,0.36,0.21,0.32,0.25,0.13,0.39,0.22,0.36
2,Bastos,ao ANG,DF,32.0,32,31,2628.0,29.2,3.0,0.0,...,0.1,0.0,0.1,0.1,0.1,0.06,0.0,0.06,0.06,0.06
3,Marlon Freitas,br BRA,MF,28.0,32,28,2425.0,26.9,0.0,5.0,...,0.0,0.19,0.19,0.0,0.19,0.06,0.16,0.22,0.06,0.22
4,Gregore,br BRA,MF,29.0,33,25,2220.0,24.7,3.0,2.0,...,0.12,0.08,0.2,0.12,0.2,0.05,0.07,0.12,0.05,0.12
5,Alexander Barboza,ar ARG,DF,28.0,26,23,2033.0,22.6,0.0,1.0,...,0.0,0.04,0.04,0.0,0.04,0.06,0.04,0.1,0.06,0.1
6,Jefferson Savarino,ve VEN,"MF,FW",27.0,28,22,2009.0,22.3,8.0,7.0,...,0.36,0.31,0.67,0.36,0.67,0.23,0.29,0.52,0.22,0.51
7,Tchê Tchê,br BRA,"MF,FW",31.0,28,16,1570.0,17.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.04,0.16,0.2,0.04,0.2
8,Júnior Santos,br BRA,FW,29.0,24,16,1481.0,16.5,4.0,1.0,...,0.24,0.06,0.3,0.24,0.3,0.38,0.09,0.47,0.38,0.47
9,Cuiabano,br BRA,DF,20.0,23,16,1476.0,16.4,4.0,1.0,...,0.24,0.06,0.3,0.24,0.3,0.11,0.08,0.19,0.11,0.19


In [84]:
gringos_df = get_gringos(teste_df, gringos_df, clube='Flamengo')

# Analysis